In [1]:
import numpy as np
import pandas as pd

# 데이터 불러오기

In [2]:
df_demo = pd.read_csv('data/LPOINT_BIG_COMP_01_DEMO.csv') # 고객정보
df_pdde = pd.read_csv('data/LPOINT_BIG_COMP_02_PDDE.csv', parse_dates = [6]) # 구매정보
df_pd = pd.read_csv('data/LPOINT_BIG_COMP_04_PD_CLAC.csv') # 상품 정보

C:\Users\Eugene\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
DF = pd.merge(df_pdde, df_demo, on='cust')
DF = pd.merge(DF, df_pd, on='pd_c')
DF = DF.loc[(DF['ages'] == '20대')&(DF['cop_c'].apply(lambda x: x in ['A02', 'A03', 'A04']))]
print(DF.shape)
DF.head()

(196262, 16)


,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,ma_fem_dv,ages,zon_hlv,pd_nm,clac_hlv_nm,clac_mcls_nm
654,M496844384,A02191496496,1,A02,A020053,PD0290,2021-05-08,17,39000.0,1,여성,20대,Z17,남성티셔츠,남성의류,남성의류상의
2574,M748140584,A02262444001,1,A02,A020080,PD0290,2021-01-08,17,20000.0,2,남성,20대,Z11,남성티셔츠,남성의류,남성의류상의
2575,M748140584,A02262444001,1,A02,A020080,PD0290,2021-01-08,17,9000.0,9,남성,20대,Z11,남성티셔츠,남성의류,남성의류상의
2576,M748140584,A02262444001,1,A02,A020080,PD0290,2021-01-08,17,900.0,9,남성,20대,Z11,남성티셔츠,남성의류,남성의류상의
3166,M395590139,A02064243742,1,A02,A020012,PD0290,2021-02-01,11,600.0,6,여성,20대,Z17,남성티셔츠,남성의류,남성의류상의


In [32]:
factor1 = ['채소', '조미료', '축산물', '과일', '수산물', '건해산물', '양곡', '병통조림', '유제품']
factor2 = ['음료', '과자', '조리식품', '대용식', '냉동식품', '냉장식품', '주류']

fa1df = DF.loc[DF['clac_hlv_nm'].apply(lambda x: x in factor1)]
fa2df = DF.loc[DF['clac_hlv_nm'].apply(lambda x: x in factor2)]

In [33]:
Factor1 = fa1df.groupby('cust')['rct_no'].unique().apply(lambda x: len(x)).to_frame('fa1_rct_n')
Factor2 = fa2df.groupby('cust')['rct_no'].unique().apply(lambda x: len(x)).to_frame('fa2_rct_n')

In [57]:
FAdf = pd.merge(Factor1, Factor2, left_index = True, right_index = True, how = 'outer')
FAdf = FAdf.fillna(0)
print(FAdf.shape)
FAdf.head()

(2138, 2)


,fa1_rct_n,fa2_rct_n
cust,,
M000871427,6.0,9.0
M001403108,12.0,12.0
M002122238,5.0,22.0
M002188008,0.0,16.0
M002238144,53.0,99.0


In [10]:
FAdf.to_csv('data/FactorAnalysis_result.csv')

## 주말 이용 비중

In [58]:
data = DF.loc[(DF['cop_c'] == 'A02')|(DF['cop_c'] == 'A03')]
data['weekday'] = data['de_dt'].dt.dayofweek
data['weekend'] = 0 # weekend 컬럼 초기화
data.loc[data['weekday'] > 4, 'weekend'] = 1 # 토, 일을 주말로
data.loc[(data['weekday'] == 4)&(data['de_hr'] > 5), 'weekend'] = 1 # 금요일 6시부터 주말
dfrct = data[['cust', 'rct_no', 'weekend']].drop_duplicates()

C:\Users\Eugene\AppData\Local\Temp\ipykernel_8692\945305570.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weekday'] = data['de_dt'].dt.dayofweek
C:\Users\Eugene\AppData\Local\Temp\ipykernel_8692\945305570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weekend'] = 0 # weekend 컬럼 초기화
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [59]:
wedf = dfrct.groupby('cust')['weekend'].agg(['sum', 'count'])
wedf['weekend_ratio'] = wedf['sum'] / wedf['count']
print(wedf.shape)
wedf.head()

(1898, 3)


,sum,count,weekend_ratio
cust,,,
M000871427,5,11,0.454545
M001403108,9,25,0.360000
M002238144,27,64,0.421875
M002352655,8,23,0.347826
M003490042,3,3,1.000000


In [61]:
df = pd.merge(FAdf, wedf[['weekend_ratio']], right_index = True, left_index = True)
df # weekend_ratio의 널 값은 2021년에 마트, 슈퍼를 이용하지 않은 고객, 비율이므로 아예 제외
   # 요인 분석 변수는 장바구니개수기 때문에 이력이 없으면 이용 횟수가 0인 것으로 처리

,fa1_rct_n,fa2_rct_n,weekend_ratio
cust,,,
M000871427,6.0,9.0,0.454545
M001403108,12.0,12.0,0.360000
M002238144,53.0,99.0,0.421875
M002352655,17.0,14.0,0.347826
M003490042,1.0,2.0,1.000000
...,...,...,...
M998129365,1.0,1.0,1.000000
M998574012,17.0,84.0,0.580645
M998600186,1.0,1.0,0.000000


In [63]:
df.to_csv('data/fa_weekend_total.csv', index_label='cust')